# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Parameter Tuning

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

# AL SER UN PROCESO MUY COSTOSO EN TIEMPO Y MEMORIA, SE GUARDAN LOS RESULTADOS FINALES EN VARIABLES Y SE COMENTA TODO EL CÓDIGO

In [ ]:
best_params_decision_tree = ''
best_params_random_forest = ''
best_params_xgboost = ''

---
---
---


In [ ]:
# import nbimporter # pip install nbimporter
# nbimporter.options['only_defs'] = False
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import GridSearchCV
# import submission_framework as SF

In [ ]:
# columnas_a_mano = ['total_checkouts_month_5',
# 'timestamp_last_checkout',
# 'timestamp_last_event',
# 'has_checkout_month_5',
# 'total_checkouts',
# 'days_to_last_event',
# 'total_checkouts_last_week',
# 'total_checkouts_months_1_to_4',
# 'total_conversions',
# 'total_session_conversions',
# 'total_events',
# 'total_sessions',
# 'avg_events_per_session',
# 'total_session_checkouts',
# 'has_checkout'
# ]

## Decision Tree

In [ ]:
# from sklearn.tree import DecisionTreeClassifier, export_graphviz

# param_grid = {#'max_depth': np.arange(3, 10),
#               'criterion':['gini','entropy'],
#               #'min_samples_leaf':np.arange(1, 5),
#               #'min_samples_split':np.arange(2, 5)
#  }

# def decision_tree_GS(X_train, y_train, seed):
#     tree = GridSearchCV(DecisionTreeClassifier(random_state=seed), param_grid, cv=5)
#     tree.fit(X_train, y_train)
#     return tree

# model,auc = SF.full_framework_wrapper('decision_tree_GS',decision_tree_GS,columns=columnas_a_mano,verbosity=-1)
# best_params_decision_tree = model.best_params_

# best_params_decision_tree

## Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# param_grid = {#'max_depth': np.arange(3, 10),
#               'n_estimators':np.arange(15,20),
#               #'n_jobs':np.arange(1,5),
#               'criterion':['gini','entropy'],
#               #'class_weight':['balanced']              
# }

# def random_forest_GS(X_train, y_train, seed):
#     rf = GridSearchCV(RandomForestClassifier(random_state=seed), param_grid, cv=5)
#     y_train.shape = y_train.shape[0]
#     rf.fit(X_train, y_train)
#     return rf

# model, auc = SF.full_framework_wrapper('random_forest_GS',random_forest_GS,columns=columnas_a_mano,verbosity=-1)
# best_params_random_forest = model.best_params_
# best_params_random_forest

## XGBoost


In [ ]:
# import xgboost as xgb #conda install -c conda-forge xgboost 

# param_grid = {'objective': ['binary:logistic'],
#               #'learning_rate':[0.1,0.3],
#               #'max_depth':np.arange(3,10,2),
#               #'min_child_weight':np.arange(1,6,2),
#               #'gamma':np.arange(0,0.5,0.1),
#               #'subsample':np.arange(0.6,1.0,0.1),
#               #'colsample_bytree':np.arange(0.6,1.0,0.1),
#               #'n_estimators':np.arange(100,105),
#               #'reg_alpha':np.arange(1,15),
#               #'colsample_bytree':np.arange(0.3,1,0.1),
# }

# def xgboost_GS(X_train, y_train, seed):
#     rf = GridSearchCV(xgb.XGBClassifier(random_state=seed), param_grid, cv=5)
#     y_train.shape = y_train.shape[0]
#     rf.fit(X_train, y_train)
#     return rf

# model, auc = SF.full_framework_wrapper('xgboost_GS',xgboost_GS,columns=columnas_a_mano,verbosity=-1)
# best_params_xgboost = model.best_params_
# best_params_xgboost

## KNN

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# model_name = 'knn'

# rango = 5
# def knn_gridsearch(x_train, y_train, x_test, y_test):
#     y_train.shape = y_train.shape[0]
#     k_max = (0,0)
#     for k in range(1,100, rango):
#         knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
#         knn.fit(x_train, y_train)
#         score = knn.score(x_test, y_test)*100
#         if score > k_max[1]:
#             k_max = (k,score)
#         print("K: {}, {}".format(k, score))
#     a = k_max[0]
#     k_max = (0,0)
#     for k in range(a - rango, a + rango):
#         knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
#         knn.fit(x_train, y_train)
#         score = knn.score(x_test, y_test)*100
#         if score > k_max[1]:
#             k_max = (k,score)
#         print("K: {}, {}".format(k, score))
#     return k_max[0]

# X, y = SF.fr1_extract_X_y(df_users, df_y)
# X_train, X_test, y_train, y_test = SF.fr2_train_test_split(X, y, seed, test_size)    
# K = knn_gridsearch(X_train, y_train, X_test, y_test)

# def knn(X_train, y_train, seed, k=K):
#     knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
#     y_train.shape = y_train.shape[0]
#     knn.fit(X_train, y_train)
#     return knn

# SF.full_framework_wrapper(f'KNN{K}',knn,columns=columnas_a_mano,verbosity=-1)